In [1]:
import pandas as pd
import seaborn as sns
import plotly.express as px
from copy import copy
import matplotlib.pyplot as plt
import numpy as np
import plotly.figure_factory as ff
import plotly.graph_objects as go


#Get the dataset 
df = pd.read_csv("stocks_dataset.csv") #data till 2020 (Includes first 6 months of covid shock for blue chips)
df.info() #SP500 is index

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1699 entries, 0 to 1698
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    1699 non-null   object 
 1   FB      1699 non-null   float64
 2   TWTR    1699 non-null   float64
 3   NFLX    1699 non-null   float64
 4   BA      1699 non-null   float64
 5   T       1699 non-null   float64
 6   MGM     1699 non-null   float64
 7   TSLA    1699 non-null   float64
 8   sp500   1699 non-null   float64
dtypes: float64(8), object(1)
memory usage: 119.6+ KB


In [2]:
# Some logic checks for the data:

# What is the average price of Netflix?
# What is the maximum price for S&P500 stock over the specified time period?

df.describe()

,FB,TWTR,NFLX,BA,T,MGM,TSLA,sp500
count,1699.000000,1699.000000,1699.000000,1699.000000,1699.000000,1699.000000,1699.000000,1699.000000
mean,135.966863,31.296698,195.423654,218.325633,35.296986,25.720865,311.299388,2413.219428
std,50.062473,11.508628,127.700712,98.968191,3.412142,5.200605,203.097452,424.825240
min,44.820000,14.010000,44.887142,95.010002,26.770000,7.140000,120.500000,1741.890015
25%,88.080002,19.600001,90.474998,132.840004,32.790001,21.935001,218.829994,2053.294921
50%,136.759995,31.610001,143.830002,158.289993,34.900002,25.879999,253.740005,2360.159912
75%,178.075005,38.229999,320.024993,334.360001,38.004999,29.250000,323.565002,2783.330078
max,268.440002,73.309998,548.729980,440.619995,43.470001,38.029999,1643.000000,3386.149902


In [12]:
%%latex

Recall the CAPM model formula

$$r_i = r_f + \beta_i(r_m-r_f)$$

<IPython.core.display.Latex object>

In [23]:
%%latex
The goal is to estimate the $$\beta_i, i \in \mathbb{R}_{++}$$

<IPython.core.display.Latex object>

In [34]:
%%latex

Beta represents the slope of the regression line between market return vs stock return. A better way to think of this is
that \beta represents the relationship between systemic risk (AKA Volatility) and expected return for an asset

<IPython.core.display.Latex object>

In [25]:
# First we need to normalize the prices based on the initial price

def normalize(df):
  x = df.copy()
  for i in x.columns[1:]:
    x[i] = x[i]/x[i][0]
  return x
df_norm = normalize(df)
df_norm

,Date,FB,TWTR,NFLX,BA,T,MGM,TSLA,sp500
0,11/7/2013,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,11/8/2013,0.999369,0.927617,1.024598,1.015056,1.001709,1.027449,0.986979,1.013428
2,11/11/2013,0.971405,0.955457,1.033807,1.007756,0.997721,1.033369,1.035272,1.014160
3,11/12/2013,0.980025,0.933185,1.021018,1.006235,1.001709,1.036598,0.985905,1.011756
4,11/13/2013,1.024180,0.948775,1.025760,1.012623,0.998861,1.060280,0.992345,1.019947
...,...,...,...,...,...,...,...,...,...
1694,8/3/2020,5.297729,0.810468,10.678394,1.233899,0.843634,0.878364,10.624597,1.885705
1695,8/4/2020,5.252944,0.809577,10.914398,1.255190,0.854742,0.899892,10.638906,1.892516
1696,8/5/2020,5.238015,0.819376,10.753136,1.325222,0.850185,0.899892,10.624740,1.904685
1697,8/6/2020,5.577796,0.839421,10.902404,1.309406,0.849900,0.993541,10.657365,1.916927


In [28]:
def interactive_plot(df_norm, title):
  fig = px.line(title = title)
  for i in df_norm.columns[1:]:
    fig.add_scatter(x = df_norm['Date'], y = df_norm[i], name = i)
  fig.show()

interactive_plot(df_norm, 'Normalized Prices')

In [32]:
# Calculate Daily Return

def daily_return(df):
  df_daily_return = df.copy()
  for i in df.columns[1:]:
        for j in range(1, len(df)):
            df_daily_return[i][j] = ((df[i][j]- df[i][j-1])/df[i][j-1]) * 100
        df_daily_return[i][0] = 0
  return df_daily_return

stocks_daily_return = daily_return(df)
stocks_daily_return

,Date,FB,TWTR,NFLX,BA,T,MGM,TSLA,sp500
0,11/7/2013,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,11/8/2013,-0.063082,-7.238307,2.459768,1.505597,0.170883,2.744887,-1.302144,1.342756
2,11/11/2013,-2.798229,3.001200,0.898778,-0.719159,-0.398064,0.576223,4.893077,0.072293
3,11/12/2013,0.887446,-2.331002,-1.237020,-0.150907,0.399655,0.312495,-4.768482,-0.237039
4,11/13/2013,4.505467,1.670635,0.464452,0.634774,-0.284328,2.284533,0.653116,0.809534
...,...,...,...,...,...,...,...,...,...
1694,8/3/2020,-0.674101,-0.027481,1.992307,2.702534,0.135230,1.429459,3.790991,0.718102
1695,8/4/2020,-0.845374,-0.109923,2.210104,1.725521,1.316674,2.450974,0.134680,0.361193
1696,8/5/2020,-0.284196,1.210462,-1.477519,5.579446,-0.533156,0.000000,-0.133153,0.642974
1697,8/6/2020,6.486835,2.446311,1.388142,-1.193483,-0.033501,10.406699,0.307062,0.642770


In [35]:
#Calculate Beta_i 

#Least squares polynomial fit
beta, alpha = np.polyfit(stocks_daily_return['sp500'], stocks_daily_return['NFLX'], 1)
print(beta,alpha*100)




1.0445724485449472 0.12722576030846008


In [36]:
# and E[r_i] 

stocks_daily_return['sp500'].mean()
rm = stocks_daily_return['sp500'].mean() * 252

#r_f ... risk free asset will be 0 for now (subject to change based on 10-y Treasury)
rf = 0 
E_R_NFLX = rf + (beta * (rm-rf)) 
E_R_NFLX

11.733213986745898

In [ ]:
#Do a loop for all stocks in port